In [1]:
import pandas as pd
import numpy as np
import nltk
import seaborn as sns
#from spellchecker import SpellChecker
import matplotlib.pyplot as plt
from collections import Counter #uso para n-gramas
#from wordcloud import WordCloud
import re # talvez regex vai ajudar na normalizaçao

Aqui pretendo verificar a frequência das palavras (devo retirar stopwords) para fazer um heatmap.
Desafio extra: subir o texto de insumo, fazer o mesmo pré-processamento e exibir somente heywords que não aparecem no texto de insumo.

In [2]:
csv_file_path = 'C:/Users/Administrator/Documents/repos/mestrado/visual_analytics/corpus_completo.csv'

# Lendo o csv como um df
df = pd.read_csv(csv_file_path)

#Criando uma cópia
df_va = df.copy()

In [3]:
df_va.groupby('nota').count()

,file_name,content
nota,,
0,25,25
1,211,211
2,628,628
3,715,715
4,477,477
5,237,237


In [4]:
nltk.download('stopwords')
nltk.download('punkt') # é um tokenizador, importante para nltk.word_tokenize
# nltk.download('rslp') é um stemmer. acho q nao vou usar
nltk.download('punkt_tab') # Precisei que devia fazer download desse pacote pq o punkt nao tava funcionando

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [5]:
# Pré-processamento

# Devo tirar acentos?? Ex: Belém
# Retirar números? Na duvida, retirei.
# Não fiz stemming visto que poderia perder erros ortográficos

#Retirada de sinais gráficos, pontuações e espaços
def clean_cp(text):
    cleaned = text.lower() #Deixando tudo minúsculo
    cleaned = re.sub('[^\w\s]', '', cleaned) # Removendo pontuacao
    #cleaned = re.sub('[0-9]+', '', cleaned) # Removendo números 
    cleaned = re.sub('\d+', '', cleaned) # Removendo números NÃO TÁ FUNCIONANDO AAAAAAAAAAAAAAAAA Começou a funcionar qdo pus o lower como primeiro comando da funçao
    cleaned = re.sub('\s+', ' ', cleaned) # Removendo espaços extras
    cleaned = re.sub('\s+', ' ', cleaned)
    return cleaned.strip() # Removendo tabs

#Retirada de stopwords.. Essa parte deve ser importante na hora de ver as palavras mais frequentes (heatmap)
#def stopwords_cp(text):
  #  stopwords = nltk.corpus.stopwords.words('portuguese')
   # tokenized = nltk.word_tokenize(text, language='portuguese')
   # sem_stopwords = [token for token in tokenized if token not in stopwords]
   # return ' '.join(sem_stopwords)

df_va['content'] = df_va['content'].apply(clean_cp)
#df_va['content'] = df_va['content'].apply(stopwords_cp)

In [6]:
df_va['content'].sample(5)

934     data de bibiana cambará para prefeitura munici...
1038    brasilia para prefeitura municipal cidade de b...
75      azulejos valiosos os casarões históricos da ci...
596     depredaçoes em belem sres prefeitura municipal...
1267    remite bibiana cambará moradora belémpara ende...
Name: content, dtype: object

In [7]:
# Tokenizando e retirando stopwords: importante para ver a frequencia das palavras

def tokenized_cp(text):
   stopwords = nltk.corpus.stopwords.words('portuguese') # Carregando as stopwords do português
   tokenized = nltk.word_tokenize(text, language='portuguese') #Transforma o texto em tokens
   text_sem_stopwords = [token for token in tokenized if token not in stopwords] # Deixando somente o que nao é stopword no texto
   return text_sem_stopwords

df_va['tokenized_content'] = df_va['content'].apply(tokenized_cp)

In [8]:
df_va['tokenized_content'].sample(5)

222     [outubro, prefeitura, municipal, hoje, quero, ...
2274    [belém, outubro, prezado, prefeito, nome, bibi...
2178    [carta, leitor, folha, belém, belém, outubro, ...
1450    [belém, setembro, senhor, b, morador, cidade, ...
1590    [belém, julho, prezados, funcionários, prefeit...
Name: tokenized_content, dtype: object

In [9]:
df_va.dtypes

file_name            object
content              object
nota                  int64
tokenized_content    object
dtype: object

In [10]:
# Contando o número de tokens SEM STOPWORDS (diferente do typestokenttr.ipynb) para cada texto: usarei isso para o gráfico 2, que demonstra o número mínimo e número máximo de tokens

# Contando tokens sem considerar a nota
df_va['num_tokens'] = df_va['tokenized_content'].apply(len)

# Agrupando de acordo com cada nota
qtde_tokens_nota = df_va.groupby('nota')['num_tokens'].sum()

In [11]:
df_va.tail()

,file_name,content,nota,tokenized_content,num_tokens
2288,20152t4p936n5r.docx,belem de outubro de senhores prefeitura munici...,5,"[belem, outubro, senhores, prefeitura, municip...",100
2289,20152t4p93r.docx,pará de outubro de ao senhor prefeito municipa...,5,"[pará, outubro, senhor, prefeito, municipal, p...",107
2290,20152t4p953n5r.docx,pilas de outubro de prefeitura municipal de be...,5,"[pilas, outubro, prefeitura, municipal, belém,...",94
2291,20152t4p96r.docx,para a prefeitura municipal de um morador de b...,5,"[prefeitura, municipal, morador, belém, senhor...",99
2292,20152t4p997n5r.docx,cidade do belém dia de outubro de exelentíssim...,5,"[cidade, belém, dia, outubro, exelentíssima, p...",145


In [12]:
# Verificando a ocorrência dos tokens

# Criando uma lista só com todos os tokens (?)
all_tokens = [token for tokens in df_va['tokenized_content'] for token in tokens]

# Contando a ocorrencia de cada token
token_counts = Counter(all_tokens)
token_counts

# Convertendo para um df para ficar melhor de visualizar
# token_counts_df = pd.DataFrame(token_counts.items(), columns=['token', 'count']).sort_values(by='count', ascending=False)

Counter({'azulejos': 8132,
         'cidade': 5561,
         'belém': 4062,
         'casarões': 4001,
         'prefeitura': 2903,
         'situação': 2385,
         'municipal': 2249,
         'roubos': 2057,
         'bibiana': 1780,
         'patrimônio': 1725,
         'vandalismo': 1694,
         'cambará': 1646,
         'medidas': 1578,
         'históricos': 1261,
         'proteção': 1239,
         'xix': 1227,
         'maior': 1226,
         'palacete': 1178,
         'problema': 1117,
         'parte': 1093,
         'variedade': 1064,
         'alvo': 1049,
         'século': 1047,
         'valor': 1031,
         'desde': 1023,
         'maria': 1013,
         'mercado': 997,
         'histórico': 990,
         'cultural': 988,
         'valiosos': 957,
         'morador': 947,
         'ano': 931,
         'ter': 902,
         'silva': 888,
         'vitor': 882,
         'atenciosamente': 859,
         'carta': 859,
         'cidades': 851,
         'porque': 847,
   

In [27]:
# Agrupando a contagem de tokens de acordo com cada nota
nota_token_counts = (
    df_va.groupby('nota')['tokenized_content']
    .apply(lambda texts: Counter([token for text in texts for token in text]))
)

# Convertendo cada token e contagem em um dataframe
df_frequency = nota_token_counts.reset_index()
df_frequency.columns = ['nota', 'token', 'token_frequency']
sorted_tokens = df_frequency.sort_values('token_frequency', ascending=False)
sorted_tokens

,nota,token,token_frequency
44443,3,azulejos,2531.0
29648,2,azulejos,2173.0
44465,3,cidade,1838.0
59238,4,azulejos,1810.0
29670,2,cidade,1383.0
...,...,...,...
87640,5,vendeuse,NaN
87642,5,comunico,NaN
87644,5,roubáos,NaN
87645,5,impeder,NaN


In [28]:
df_words = pd.DataFrame()

In [29]:
# Agrupando a contagem de tokens por nota de forma a mostrar os 15 mais comuns para cada nota ( 15 * 6 = 90 linhas portanto )
df_words = top_tokens_per_grade
top_tokens_per_grade = (
    sorted_tokens.groupby('nota')
    .head(5)  # Select the top 15 tokens per grade
    .reset_index(drop=True)
)

In [40]:
df_words

,nota,token,token_frequency
0,3,azulejos,2531.0
1,2,azulejos,2173.0
2,3,cidade,1838.0
3,4,azulejos,1810.0
4,2,cidade,1383.0
...,...,...,...
115,0,fazer,8.0
116,0,problema,8.0
117,0,ano,8.0
118,0,parte,8.0


In [51]:
df_words_sorted = (df_words.sort_values(by=['nota', 'token_frequency'], ascending=[True, False]))
df_words_0 = df_words_sorted.loc[df_words_sorted['nota'] == 0, :]
df_words_1 = df_words_sorted.loc[df_words_sorted['nota'] == 1, :]
df_words_2 = df_words_sorted.loc[df_words_sorted['nota'] == 2, :]
df_words_3 = df_words_sorted.loc[df_words_sorted['nota'] == 3, :]
df_words_4 = df_words_sorted.loc[df_words_sorted['nota'] == 4, :]
df_words_5 = df_words_sorted.loc[df_words_sorted['nota'] == 5, :]
df_words_0 

,nota,token,token_frequency
100,0,azulejos,49.0
101,0,cidade,21.0
102,0,prefeitura,18.0
103,0,valiosos,13.0
104,0,casarões,13.0
105,0,situação,12.0
106,0,municipal,11.0
107,0,capital,10.0
108,0,pessoas,10.0
109,0,xix,10.0


In [58]:
# Nota 0  >>>>> QUEM SABE USAR plotly.subplots SEJA UMA BOAAA <<<<<< 
import plotly.graph_objects as go
import pandas as pd


# Pivot the data for heatmap
heatmap_data = df_words_0.pivot(index='token', columns='nota', values='token_frequency').fillna(0)

# Create the heatmap
fig = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,  # Frequency matrix
    x=heatmap_data.columns,  # Grades (x-axis)
    y=heatmap_data.index,    # Tokens (y-axis)
    colorscale='Reds',       # Heatmap color scale
    colorbar=dict(title="Frequência"),  # Colorbar title
    hoverongaps=False,       # Ensure no gaps show on hover
    hovertemplate=(
        "Token: %{y}<br>" + #Exibir título pelo Streamlit, q aí fica título pra cada coluna
        "Frequência: %{z}<extra></extra>"
    )
))

# Update layout for better visualization
fig.update_layout(
   # title="20 Tokens Mais Frequentes", >> Por título no streamlit
    height=500,  # Adjust height to make the heatmaps narrow
    width=300,  # Adjust width for each heatmap ....  * len(grades)
    template="plotly_white",
    plot_bgcolor="rgba(0, 0, 0, 0)",  # Transparent plot background
    paper_bgcolor="rgba(0, 0, 0, 0)",  # Transparent outer background
    font=dict(color="white")  # White font for all text
)
#fig.update_yaxes(title="Tokens", automargin=True)
#fig.update_xaxes(title="", showticklabels=False)

# Show the figure
fig.show()

In [59]:
import plotly.graph_objects as go
import pandas as pd

# Example structure for df_words (replace with your actual data)
# df_words = pd.DataFrame({
#     'nota': [0, 0, 0, 1, 1, 1, 2, 2, 2],
#     'token': ['word1', 'word2', 'word3', 'word4', 'word5', 'word6', 'word7', 'word8', 'word9'],
#     'token_frequency': [50, 40, 30, 45, 35, 25, 60, 50, 40]
# })

# Sort tokens within each grade based on frequency
df_words_sorted = (
    df_words
    .sort_values(by=['nota', 'token_frequency'], ascending=[True, False])
)

# List of unique grades
grades = df_words_sorted['nota'].unique()

# Generate a heatmap for each grade
for grade in grades:
    # Filter data for the specific grade
    grade_data = df_words_sorted[df_words_sorted['nota'] == grade]
    
    # Tokens and frequencies
    tokens = grade_data['token']
    frequencies = grade_data['token_frequency']
    
    # Create the heatmap
    fig = go.Figure(data=go.Heatmap(
        z=[frequencies],  # Heatmap values
        x=[grade],        # Grade label as a single column
        y=tokens,         # Tokens for the y-axis
        colorscale='Reds',
        showscale=False,  # Remove the colorbar
        hovertemplate=(
            "Nota: %{x}<br>" +
            "Token: %{y}<br>" +
            "Frequência: %{z}<extra></extra>"
        )
    ))

    # Update layout for better visualization
    fig.update_layout(
        title=f"Tokens Mais Frequentes para Nota {grade}",
        height=500,  # Adjust height to make the heatmap narrow
        width=300,   # Adjust width for each heatmap
        template="plotly_white",
        font=dict(color="black"),
        plot_bgcolor="rgba(0, 0, 0, 0)",  # Transparent plot background
        paper_bgcolor="rgba(0, 0, 0, 0)",  # Transparent outer background
    )
    fig.update_yaxes(title="Tokens", automargin=True)
    fig.update_xaxes(title="", showticklabels=False)

    # Show the figure
    fig.show()


In [ ]:
# Nota 0  >>>>> QUEM SABE USAR plotly.subplots SEJA UMA BOAAA <<<<<< BACKUP
import plotly.graph_objects as go
import pandas as pd


# Pivot the data for heatmap
heatmap_data = df_words_0.pivot(index='token', columns='nota', values='token_frequency').fillna(0)

# Create the heatmap
fig = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,  # Frequency matrix
    x=heatmap_data.columns,  # Grades (x-axis)
    y=heatmap_data.index,    # Tokens (y-axis)
    colorscale='Reds',       # Heatmap color scale
    colorbar=dict(title="Frequência"),  # Colorbar title
    hoverongaps=False,       # Ensure no gaps show on hover
    hovertemplate=(
        "Nota: %{x}<br>" +
        "Token: %{y}<br>" +
        "Frequência: %{z}<extra></extra>"
    )
))

# Update layout for better visualization
fig.update_layout(
   # title="20 Tokens Mais Frequentes", >> Por título no streamlit
    height=500,  # Adjust height to make the heatmaps narrow
    width=300,  # Adjust width for each heatmap ....  * len(grades)
    template="plotly_white",
    plot_bgcolor="rgba(0, 0, 0, 0)",  # Transparent plot background
    paper_bgcolor="rgba(0, 0, 0, 0)",  # Transparent outer background
    font=dict(color="white")  # White font for all text
)

# Show the figure
fig.show()

In [52]:
import plotly.graph_objects as go # QUEM SABE ISSO SEJA UTIL SEI LAAAAA
from plotly.subplots import make_subplots
import pandas as pd

# Example structure for df_words (replace with your actual data)
# df_words = pd.DataFrame({
#     'nota': [0, 0, 0, 1, 1, 1, 2, 2, 2],
#     'token': ['word1', 'word2', 'word3', 'word4', 'word5', 'word6', 'word7', 'word8', 'word9'],
#     'token_frequency': [50, 40, 30, 45, 35, 25, 60, 50, 40]
# })

# Sort tokens within each grade based on frequency
df_words_sorted = (
    df_words_0
    .sort_values(by=['nota', 'token_frequency'], ascending=[True, False])
)

# List of unique grades
grades = df_words_sorted['nota'].unique()

# Create subplots with one row and as many columns as there are grades
fig = make_subplots(
    rows=1,
    cols=len(grades),
    subplot_titles=[f"Nota {grade}" for grade in grades],
    shared_yaxes=True  # Share y-axis for consistent token display
)

# Loop through each grade to create individual heatmaps
for i, grade in enumerate(grades, start=1):
    # Filter data for the specific grade
    grade_data = df_words_sorted[df_words_sorted['nota'] == grade]
    
    # Tokens and frequencies
    tokens = grade_data['token']
    frequencies = grade_data['token_frequency']
    
    # Add a heatmap for the grade
    fig.add_trace(
        go.Heatmap(
            z=[frequencies],  # Heatmap values
            x=[grade],        # Grade label as a single column
            y=tokens,         # Tokens for the y-axis
            colorscale='Reds',
            colorbar=dict(title="Frequência") if i == len(grades) else None,  # Single colorbar for the last subplot
            hovertemplate=(
                "Nota: %{x}<br>" +
                "Token: %{y}<br>" +
                "Frequência: %{z}<extra></extra>"
            )
        ),
        row=1, col=i
    )

# Update layout for better visualization
fig.update_layout(
    title="Tokens Mais Frequentes por Nota",
    height=500,  # Adjust height to make the heatmaps narrow
    width=300 * len(grades),  # Adjust width for each heatmap
    template="plotly_white",
    font=dict(color="black"),
    plot_bgcolor="rgba(0, 0, 0, 0)",  # Transparent plot background
    paper_bgcolor="rgba(0, 0, 0, 0)",  # Transparent outer background
)

# Adjust individual subplot properties
fig.update_yaxes(title="Tokens", automargin=True)
fig.update_xaxes(title="", showticklabels=False)

# Show the figure
fig.show()


In [36]:
import plotly.graph_objects as go
import pandas as pd

# Example structure for df_words (replace with your actual data)
# df_words = pd.DataFrame({
#     'nota': [0, 0, 0, 1, 1, 1, 2, 2, 2],
#     'token': ['word1', 'word2', 'word3', 'word4', 'word5', 'word6', 'word7', 'word8', 'word9'],
#     'token_frequency': [50, 40, 30, 45, 35, 25, 60, 50, 40]
# })

# Pivot the data for heatmap
heatmap_data = df_words.pivot(index='token', columns='nota', values='token_frequency').fillna(0)

# Create the heatmap
fig = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,  # Frequency matrix
    x=heatmap_data.columns,  # Grades (x-axis)
    y=heatmap_data.index,    # Tokens (y-axis)
    colorscale='Reds',       # Heatmap color scale
    colorbar=dict(title="Frequência"),  # Colorbar title
    hoverongaps=False,       # Ensure no gaps show on hover
    hovertemplate=(
        "Nota: %{x}<br>" +
        "Token: %{y}<br>" +
        "Frequência: %{z}<extra></extra>"
    )
))

# Update layout for better visualization
fig.update_layout(
    title="20 Tokens Mais Frequentes por Nota",
    xaxis_title="Nota",
    yaxis_title="Tokens",
    xaxis=dict(
        tickvals=heatmap_data.columns,  # Ensure separation between grades
        title_font=dict(size=14, color="white"),
        tickfont=dict(color="white")
    ),
    yaxis=dict(
        title_font=dict(size=14, color="white"),
        tickfont=dict(color="white"),
        automargin=True
    ),
    template="plotly_white",
    plot_bgcolor="rgba(0, 0, 0, 0)",  # Transparent plot background
    paper_bgcolor="rgba(0, 0, 0, 0)",  # Transparent outer background
    font=dict(color="white")  # White font for all text
)

# Show the figure
fig.show()


In [38]:
import plotly.graph_objects as go
import pandas as pd

# Example structure for df_words (replace with your actual data)
# df_words = pd.DataFrame({
#     'nota': [0, 0, 0, 1, 1, 1, 2, 2, 2],
#     'token': ['word1', 'word2', 'word3', 'word4', 'word5', 'word6', 'word7', 'word8', 'word9'],
#     'token_frequency': [50, 40, 30, 45, 35, 25, 60, 50, 40]
# })

# Sort tokens by frequency within each grade
df_words_sorted = (
    df_words
    .sort_values(by=['nota', 'token_frequency'], ascending=[True, False])
)

# Create a composite column to ensure distinct tokens per grade
df_words_sorted['token_nota'] = df_words_sorted['nota'].astype(str) + '|' + df_words_sorted['token']

# Pivot the data for heatmap
heatmap_data = df_words_sorted.pivot(index='token_nota', columns='nota', values='token_frequency').fillna(0)

# Update y-axis tokens for better display (show only token in hover)
heatmap_data.index = heatmap_data.index.str.split('|').str[1]

# Create the heatmap
fig = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,  # Frequency matrix
    x=heatmap_data.columns,  # Grades (x-axis)
    y=heatmap_data.index,    # Tokens (y-axis)
    colorscale='Reds',       # Heatmap color scale
    colorbar=dict(title="Frequência"),  # Colorbar title
    hovertemplate=(
        "Nota: %{x}<br>" +
        "Token: %{y}<br>" +
        "Frequência: %{z}<extra></extra>"
    )
))

# Update layout for better visualization
fig.update_layout(
    title="20 Tokens Mais Frequentes por Nota (Ordenados)",
    xaxis_title="Nota",
    yaxis_title="Tokens (Ordenados por Frequência)",
    xaxis=dict(
        tickvals=heatmap_data.columns,  # Ensure separation between grades
        title_font=dict(size=14, color="white"),
        tickfont=dict(color="white")
    ),
    yaxis=dict(
        title_font=dict(size=14, color="white"),
        tickfont=dict(color="white"),
        automargin=True
    ),
    template="plotly_white",
    plot_bgcolor="rgba(0, 0, 0, 0)",  # Transparent plot background
    paper_bgcolor="rgba(0, 0, 0, 0)",  # Transparent outer background
    font=dict(color="white")  # White font for all text
)

# Show the figure
fig.show()


In [54]:
# ISSO NAO TÁ EXIBINDO NADA MAS QUEM SABE É UMA BOA IDÉIA PRA JANEIRO

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Example structure for df_words (replace with your actual data)
# df_words = pd.DataFrame({
#     'nota': [0, 0, 0, 1, 1, 1, 2, 2, 2],
#     'token': ['word1', 'word2', 'word3', 'word4', 'word5', 'word6', 'word7', 'word8', 'word9'],
#     'token_frequency': [50, 40, 30, 45, 35, 25, 60, 50, 40]
# })

# Sort tokens within each grade based on frequency
df_words_sorted = (
    df_words
    .sort_values(by=['nota', 'token_frequency'], ascending=[True, False])
)

# List of unique grades
grades = df_words_sorted['nota'].unique()

# Create subplots with one row and as many columns as there are grades
fig = make_subplots(
    rows=1,
    cols=len(grades),
    subplot_titles=[f"Nota {grade}" for grade in grades],
    shared_yaxes=True  # Share y-axis for consistent token display
)

# Loop through each grade to create individual heatmaps
for i, grade in enumerate(grades, start=1):
    # Filter data for the specific grade
    grade_data = df_words_sorted[df_words_sorted['nota'] == grade]
    
    # Tokens and frequencies
    tokens = grade_data['token']
    frequencies = grade_data['token_frequency']
    
    # Add a heatmap for the grade
    fig.add_trace(
        go.Heatmap(
            z=[frequencies],  # Heatmap values
            x=[grade],        # Grade label as a single column
            y=tokens,         # Tokens for the y-axis
            colorscale='Reds',
            colorbar=dict(title="Frequência") if i == len(grades) else None,  # Single colorbar for the last subplot
            hovertemplate=(
                "Nota: %{x}<br>" +
                "Token: %{y}<br>" +
                "Frequência: %{z}<extra></extra>"
            )
        ),
        row=1, col=i
    )

# Update layout for better visualization
fig.update_layout(
    title="Tokens Mais Frequentes por Nota",
    height=500,  # Adjust height to make the heatmaps narrow
    width=300 * len(grades),  # Adjust width for each heatmap
    template="plotly_white",
    font=dict(color="black"),
    plot_bgcolor="rgba(0, 0, 0, 0)",  # Transparent plot background
    paper_bgcolor="rgba(0, 0, 0, 0)",  # Transparent outer background
)

# Adjust individual subplot properties
fig.update_yaxes(title="Tokens", automargin=True)
fig.update_xaxes(title="", showticklabels=False)

# Show the figure
fig.show()
